# Pierre's modelization notebook

## Import of modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

## Analysis of features and target

#### Opening of the DataFrame

In [2]:
df = pd.read_csv('../../data/preprocessed_data.csv')
df.head()

,C concentration (weight%),Si concentration (weight%),Mn concentration (weight%),S concentration (weight%),P concentration (weight%),V concentration (weight%),O concentration (ppm/weight),Ti concentration (ppm/weight),N concentration (ppm/weight),Al concentration (ppm/weight),...,electrode positive,Type of weld_GMAA,Type of weld_GTAA,Type of weld_MMA,Type of weld_NGGMA,Type of weld_NGSAW,Type of weld_SA,Type of weld_SAA,Type of weld_ShMA,Type of weld_TSA
0,-1.794624,-0.241903,-1.474090,-0.150353,-0.062002,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
1,-1.794624,-0.241903,-1.474090,-0.150353,-0.062002,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
2,-1.794624,-0.157085,-0.467487,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
3,-1.794624,-0.157085,-0.467487,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False
4,-1.474696,0.182188,0.592095,-0.234530,0.034196,1.542981e-17,-1.143271e-16,0.0,0.0,8.694071e-17,...,True,False,False,True,False,False,False,False,False,False


#### Deletion of the useless columns for the prediction and determination of the best target columns

We have various steel, yet the charpy temperature seems to have huge differences depending of the type.

In [3]:
# Drop the columns that are not needed
df = df.drop('Charpy temperature (deg C)',axis=1)

# Get the name of the target columns 
testNames = ['Yield strength (MPa)','Ultimate tensile strength (MPa)', 'Elongation (%)',
            'Reduction of Area (%)','Charpy impact toughness (J)']

# Get the name of the feature columns
featureNames = df.columns.difference(testNames)

### Information printing
This part is based on Clarisse's work.

This part will be usefull to decide how the target will be created with the columns referring to the welding test quality (Can be called "test columns").

We start with a Boolean columns creation : see for each row if the test is done.

In [4]:
# Initialize the dataframe that will contain the new columns
dfAugmented = df.copy()

# Initialize the list which will contain the name of the new columns
newColNames=['nb']

# Create a column that will tell how many test each observation has
# Initialize all the values to 0
dfAugmented['nb'] = 0

# Create a column, composed of boolean, with the suffix "_test" for each test column
# to see if the test is done or not (blank cells or not) and a column that will tell
# how many test each observation has
for c in testNames:
    n=c+'_test'
    dfAugmented[n] = np.where(dfAugmented[c].isna(),0,1)
    newColNames.append(n)
    dfAugmented['nb'] = dfAugmented['nb'] + dfAugmented[n]

In [5]:
# Display of test completion rate
for k in newColNames[1:]:
    dfInter = dfAugmented[k].value_counts().reset_index()
    dfInter['count'] = (dfInter['count']*100)/dfInter['count'].sum()
    print(k,'\nTest not done',int(dfInter['count'].to_list()[0]), '%\nTest done',int(dfInter['count'].to_list()[1]),'%\n')

# Display the number of tests performed on each observation
dfAugmented['nb'].value_counts()

Yield strength (MPa)_test 
Test not done 52 %
Test done 47 %

Ultimate tensile strength (MPa)_test 
Test not done 50 %
Test done 49 %

Elongation (%)_test 
Test not done 53 %
Test done 46 %

Reduction of Area (%)_test 
Test not done 52 %
Test done 47 %

Charpy impact toughness (J)_test 
Test not done 50 %
Test done 49 %



nb
1    650
4    510
5    134
0     82
3     52
2     35
Name: count, dtype: int64

Conclusion : 

Nearly half of the instances have only one test. All tests are done approximately 50% of the time. There are only 82 instances out of 1652 which do not have any tests.

### Preparation of the columns that are use to create the target 
Due to the fact that there is less than 5% of the dataset which has no test done. I choose to drop these instances.

In [6]:
# Drop the values where no test was done
dfAugmented = dfAugmented.drop(dfAugmented[dfAugmented['nb'] == 0].index)

# Verify if only 82 instances have been dropped
print(f"Number of dropped instances : {df.shape[0]-dfAugmented.shape[0]}")

Number of dropped instances : 82


Limit values have been obtained from ChatGPT to establish if a welding is good.

The first part of the work is to get the threshold and to normalize them based on the data "welddb.data".

The final dataset for the model will be created with a column called "y" which will be composed of a bool for each instance (1 if the welding is good, 0 otherwise) and the other columns of df without the columns about the tests. 

In [7]:
# Reading the data file with space-separated values and no headers
data = pd.read_table("../../data/welddb.data", sep=" ", header=None)

# Replacing all occurrences of 'N' with None to handle missing values
data.replace('N', None, inplace=True)

# Assigning column names to the dataset
data.columns = ["C concentration (weight%)", "Si concentration (weight%)", "Mn concentration (weight%)", 
                "S concentration (weight%)", "P concentration (weight%)", "Ni concentration (weight%)", 
                "Cr concentration (weight%)", "Mo concentration (weight%)", "V concentration (weight%)", 
                "Cu concentration (weight%)", "Co concentration (weight%)", "W concentration (weight%)", 
                "O concentration (ppm/weight)", "Ti concentration (ppm/weight)", "N concentration (ppm/weight)", 
                "Al concentration (ppm/weight)", "B concentration (ppm/weight)", "Nb concentration (ppm/weight)", 
                "Sn concentration (ppm/weight)", "As concentration (ppm/weight)", "Sb concentration (ppm/weight)", 
                "Current (A)", "Voltage (V)", "AC or DC", "Electrode positive or negative", 
                "Heat input (kJ/mm)", "Interpass temperature (deg C)", "Type of weld", 
                "Post weld heat treatment temperature (deg C)", "Post weld heat treatment time (hours)", 
                "Yield strength (MPa)", "Ultimate tensile strength (MPa)", "Elongation (%)", 
                "Reduction of Area (%)", "Charpy temperature (deg C)", "Charpy impact toughness (J)", 
                "Hardness (kg/mm2)", "50 FATT", "Primary ferrite in microstructure (%)", 
                "Ferrite with second phase (%)", "Acicular ferrite (%)", "Martensite(%)", 
                "Ferrite with carbide aggregate (%)", "Weld ID"]

# Removing '<' symbols from the data using regex replacement
data = data.replace("<", "", regex=True)

# Cleaning up the 'N concentration (ppm/weight)' column by splitting on 'tot' and keeping only the first part
data['N concentration (ppm/weight)'] = data['N concentration (ppm/weight)'].str.split("tot").str[0]

# Cleaning the 'Hardness (kg/mm2)' column by removing anything after '(' or 'H'
data['Hardness (kg/mm2)'] = data['Hardness (kg/mm2)'].str.split("(").str[0]
data['Hardness (kg/mm2)'] = data['Hardness (kg/mm2)'].str.split("H").str[0]

# Replacing the temperature range '150-200' with its average value '175'
data['Interpass temperature (deg C)'] = data['Interpass temperature (deg C)'].replace('150-200', '175')

# Converting all columns to float type, except columns with indices 23, 24, 27, and 43
for i, column in enumerate(data.columns):
    if i not in [23, 24, 27, 43]:
        data[column] = data[column].astype(float)

# Converting column names to strings (in case they were not already strings)
data.columns = data.columns.map(str)

# Selecting only a subset of columns, stored in 'col_test' (which is assumed to be predefined elsewhere)
data = data[testNames]

# Calculating the mean of each column in the selected subset
means = data.mean()

# Calculating the standard deviation of each column in the selected subset
stds = data.std()


# Init a dictionnary thresholds
thresholds = {
    'Yield strength (MPa)' : {350:[]},
    'Ultimate tensile strength (MPa)' : {500:[]},
    'Elongation (%)': {20:[]},
    'Reduction of Area (%)':{20:[]},
    'Charpy impact toughness (J)':{50:[]}
}

# Normalize thresholds
for c in testNames:
    val = list(thresholds[c].keys())[0]
    thresholds[c] = (val - means[c]) / stds[c]

### Prediction
Method for the prediction :

-   Create 5 datasets (one by test)
-   Create a matrix of feature Xi et a matrix for the target yi for each test
-   Drop instances that do not have target value
-   Split Xi and yi into values for training and values for test (ratio 70:30)
-   Train the RandomForestRegressor
-   Test The model
-   Apply the model on the data 
-   Deduce the good and bad welding based on the thresholds

In [12]:
# Define the hyperparameter grid to search
paramGrid = {
    'n_estimators': [50, 100, 200],              # Number of trees in the forest
    'max_depth': [None, 10, 20],                 # Maximum depth of the tree
    'min_samples_split': [2, 5],                 # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],               # Minimum number of samples required to be at a leaf node
    'bootstrap': [True, False]                   # Whether bootstrap samples are used when building trees
}

dfFinal = dfAugmented.copy()

# Create a loop over each test to train a RandomForestRegressor with Grid Search and fill missing values
for test in testNames:
    print(f"\nProcessing for test: {test}")
    
    # Create dataset for the current test, removing rows where the target value is NaN
    dfTest = dfAugmented.dropna(subset=[test], inplace=False)
    
    # Create feature matrix Xi and target vector yi
    # Drop columns that are in testNames, columns that end with '_test', and the column 'nb'
    columnsToRemove = testNames + [col for col in dfTest.columns if col.endswith('_test')] + ['nb']
    Xi = dfTest.drop(columns=columnsToRemove, inplace=False)  # Drop all specified columns
    yi = dfTest[test]  # The target is the current test
    
    # Split data into training and test sets (70% training, 30% testing)
    X_train, X_test, y_train, y_test = train_test_split(Xi, yi, test_size=0.3, random_state=42)
    
    # Setup GridSearchCV
    rf = RandomForestRegressor(random_state=42)
    
    # Initialize the GridSearchCV with cross-validation
    gridSearch = GridSearchCV(estimator=rf, param_grid=paramGrid, 
                               cv=3, scoring='neg_mean_squared_error', 
                               verbose=1, n_jobs=1)
    
    # Train the best model found by Grid Search
    gridSearch.fit(X_train, y_train)
    
    best_model = gridSearch.best_estimator_
    print(f"Best parameters for {test}: {gridSearch.best_params_}")
    
    # **Evaluate the model using MSE on the test set**
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE for {test} on test set: {mse:.4f}")
    
    # Predict the missing values in the original dataset (dfAugmented)
    # For the rows in dfAugmented where the value of 'test' is NaN, we use the model to predict the missing values
    missing_idx = dfAugmented[dfAugmented[test].isna()].index  # Get indices where the test value is NaN
    Xi_missing = dfAugmented.loc[missing_idx].drop(columns=columnsToRemove, inplace=False)  # Use the same features as before
    
    if len(Xi_missing) > 0:
        predicted_values = best_model.predict(Xi_missing)  # Predict missing values
        
        # Fill the missing values in dfAugmented with the predicted values
        dfFinal.loc[missing_idx, test] = predicted_values
        print(f"Predicted missing values for {test}: {len(missing_idx)} rows filled.")
    else:
        print(f"No missing values to predict for {test}.")





Processing for test: Yield strength (MPa)
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters for Yield strength (MPa): {'bootstrap': True, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MSE for Yield strength (MPa) on test set: 0.3005
Predicted missing values for Yield strength (MPa): 619 rows filled.

Processing for test: Ultimate tensile strength (MPa)
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters for Ultimate tensile strength (MPa): {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
MSE for Ultimate tensile strength (MPa) on test set: 0.4131
Predicted missing values for Ultimate tensile strength (MPa): 661 rows filled.

Processing for test: Elongation (%)
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters for Elongation (%): {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_spl

In [10]:
# Step 5: Once all test values are filled, deduce the quality of the welds
# We check if all test values are above their respective thresholds to decide if the weld is "Good" (1) or "Not Good" (0)
dfFinal['weld_quality'] = 1  # Initialize all to "Good" (1)

for test in testNames:
    threshold_value = thresholds[test]
    dfFinal['weld_quality'] = np.where(dfFinal[test] >= threshold_value, dfFinal['weld_quality'], 0)

# Output a summary of the weld quality
good_welds = (dfFinal['weld_quality'] == 1).sum()
bad_welds = (dfFinal['weld_quality'] == 0).sum()

print(f"Final count of good welds: {good_welds}")
print(f"Final count of bad welds: {bad_welds}")


Final count of good welds: 923
Final count of bad welds: 458
